<a href="https://colab.research.google.com/github/CrystalWang1225/ECE-471-Frequentist-Machine-Learning/blob/main/ML3_Crystal%26Yingzhi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Model Assesment and selection
Crystal & Yingzhi

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
from random import *
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
import warnings

warnings.filterwarnings("ignore")

N = 50
p = 5000

# Incorrect Way of Cross-Validation

Both incorrect and correct way of cross-validation is from 7.10

In [ ]:
# The number of simulation we need to run
n_simulation = 50
# The initialization for the accuracy for each simulation
cv_score_wrong = np.zeros((n_simulation,5))
for n in range(n_simulation):
  # predictor consists of N * p and it's pure Gaussian noise
  predictor = np.zeros((N, p))
  # labels are random 0s and 1s
  labels = np.zeros((N, 1)) 
  for each in range(N):
      labels[each] = randint(0, 1)
      predictor[each] = np.random.normal(0, 1, p)
  # the initialization of correlations of each predictor
  cor = np.zeros((p,1))

  for i in range(p):
    x = predictor[:, i].reshape(50, 1)
    cor[i] = np.corrcoef(x.T, labels.T)[1,0]

  # sort the correlation matrix and take the last 100 as the best predictors
  sorted = np.sort(cor, axis = 0)
  sorted = sorted[4900:]
  # the initialization of the indices of the best predictors
  indexBest = np.zeros((100,1))
  for j in range(100):
    for k in range(p):
      if cor[k] == sorted[j]:
        indexBest[j] = k
  index = indexBest.astype(int)
  # the initialziation of the best 100 predictor matrix
  bestPredictor = np.zeros((50,100))
  for i in range(100):
    bestPredictor[:,i] = predictor[:,index[i]].ravel()
  #using the sci-kit with n neighbors = 1
  incorrectNeighbor = KNeighborsClassifier(n_neighbors = 1)
  cv_score_wrong[n,:] = cross_val_score(incorrectNeighbor, bestPredictor, labels)

# take the average of the accuracy
averageFold = sum(cv_score_wrong)/len(cv_score_wrong)
accuracyWrong = sum(averageFold)/len(averageFold)

print("Accuracy using the wrong method: ", accuracyWrong)


Accuracy using the wrong method:  0.9711999999999998


It has demonstrates the overftting behavior of the wrong model.The screening of the prdictors consists of all the samples. Selecting the predictors after all the samples have seen have caused such a huge probelm 

# Correct Way of cross-validation

In [ ]:
n_simulation = 50
score_correct = np.zeros((1, n_simulation))
for n in range(n_simulation):
    predictor_raw = np.zeros((N, p))
    predictor = list()
    labels = np.zeros((N, 1))
    scores = np.zeros((1, 5))
    for each in range(N):
        labels[each] = randint(0, 1)
        predictor_raw[each] = np.random.normal(0, 1, p)
    # Split training data into five parts
    for i in range(5):
        predictor.append(predictor_raw[i*10:(i+1)*10, :])
    predictor = np.array(predictor)
    # Loop through each part as validation set.
    for k in range(5):
        validation = predictor[k, :, :]
        validation_label = labels[k*10:(k+1)*10, :]
        training_indexes = list(range(5))
        training_indexes.remove(k)
        training_label = np.vstack((labels[:k*10, :], labels[(k+1)*10:, :]))
        training = np.vstack(tuple([predictor[j, :, :] for j in training_indexes]))
        cor = np.zeros((p, 1))

        for i in range(p):
            x = training[:, i].reshape(40, 1)
            cor[i] = np.corrcoef(x.T, training_label.T)[1, 0]

        sorted = np.sort(cor, axis=0)
        sorted = sorted[4900:]
        index_best = np.zeros((100, 1))
        for j in range(100):
            for b in range(p):
                if cor[b] == sorted[j]:
                    index_best[j] = b
        index = index_best.astype(int)
        best_predictor = np.zeros((40, 100))
        best_predictor_val = np.zeros((10, 100))
        for i in range(100):
            best_predictor[:, i] = training[:, index[i]].ravel()
            best_predictor_val[:, i] = validation[:, index[i]].ravel()
        # Use multi-variable logistic regression to do classification
        clf = LogisticRegression(random_state=0).fit(best_predictor, training_label)
        score = clf.score(best_predictor_val, validation_label)
        scores[0, k] = score
    score_correct[0, n] = np.mean(scores)
print("Accuracy using the correct way of cross-validation: ", np.mean(score_correct))


Accuracy:  0.5076


The correct method yields an accuracy of 50%, which matches the true error rate.